# Preparations

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import torch
import numpy as np
import nibabel as nib
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import nibabel as nib

# import other scripts
import processing.preprocess_patients as from_preprocess_patients
import patients.patients as from_patients
import analysis.bayesian_optimization_plot as from_bayesian_optimization_plot
import analysis.simplot as from_simplot
import analysis.patientplot as from_patientplot
import analysis.patient_populationA_plot as from_patient_populationA_plot


# Simulations

## Model selection

In [1]:
BOplt = from_bayesian_optimization_plot.BOplt()

BOplt.plot_BO_individual_models(num_bvals=11, learning='sup')
BOplt.plot_BO_individual_models(num_bvals=5, learning='sup')
BOplt.plot_BO_individual_models(num_bvals=4, learning='sup')
BOplt.plot_BO_individual_models(num_bvals=11, learning='slf')
BOplt.plot_BO_individual_models(num_bvals=5, learning='slf')
BOplt.plot_BO_individual_models(num_bvals=4, learning='slf')
BOplt.plot_BO_all_models()

BOplt.plot_BO_individual_models_allinone()

## Model evaluation

In [ ]:
# User specific input
snrs = np.array([8, 10, 12, 15, 20, 25, 35, 50, 75, 100])
num_bvals = 11
reps = 25
num_load = 100000
exp_name = f'r{reps}_optim_snr20_d0509'

#################################################################

if num_bvals == 4:
    bvals = np.array([0, 50, 100, 800])
elif num_bvals == 5:
    bvals = np.array([0, 50, 100, 200, 800])
elif num_bvals == 11:
    bvals = np.array([0, 10, 20, 30, 40, 50, 60, 100, 200, 500, 800])

simplt_bvalset = from_simplot.Simplt_bvalset(snrs, bvals, reps, exp_name, num_load)
simplt_bvalset.plot_sim_stat()
simplt_bvalset.plot_pred_vs_truth()
simplt_bvalset.print_sim_stats()
simplt_bvalset.plot_param_space()

simplt_allbvalsets = from_simplot.Simplt_allbvalsets(snrs, reps, exp_name)
simplt_allbvalsets.plt_signal_rmse('estimated_generalisation_signal_rmse')
simplt_allbvalsets.plt_signal_rmse('estimated_expected_estimation_signal_rmse')

# EMINIENCE A

## Preprocessing

In [ ]:
def preprocess_patient_dataA():
    # user specific parameters
    patient_ids = ['EMIN_1001', 'EMIN_1005', 'EMIN_1007', 'EMIN_1011', 'EMIN_1019',
                   'EMIN_1020', 'EMIN_1022', 'EMIN_1032', 'EMIN_1038', 'EMIN_1042', 
                   'EMIN_1044', 'EMIN_1045', 'EMIN_1048', 'EMIN_1055', 'EMIN_1057', 
                   'EMIN_1060', 'EMIN_1064', 'EMIN_1066', 'EMIN_1068', 'EMIN_1075', 
                   'EMIN_1077', 'EMIN_1079', 'EMIN_1081', 'EMIN_1084', 'EMIN_1086',
                   'EMIN_1090', 'EMIN_1092', 'EMIN_1093', 'EMIN_1096', 'EMIN_1097', 
                   'EMIN_1099']
    mask_root_names = ['GTVn']
    
    # preprocessing
    for patient_id in patient_ids:
        dir_preprocessed_patient = f'../dataA/preprocessed/{patient_id}'
        dir_raw_dwi = glob.glob(f'../dataA/raw/{patient_id}/MRIdata/EP2D_STIR_DIFF_TRA_2_09I_5B-VERDIER_TRACEW_*')[0]
        
        preprocess = from_preprocess_patients.Preprocess(patient_id, dir_preprocessed_patient, dir_raw_dwi)
        preprocess.run_preprocessing()
        
        mask_names = []
        for mask_root_name in mask_root_names:
            fnames_mask = glob.glob(f'../dataA/raw/{patient_id}/Segmentation/mask_{mask_root_name}' + '*.nii.gz')

            for fname_mask in fnames_mask: 
                mask_name = fname_mask.split(f'../dataA/raw/{patient_id}/Segmentation/mask_')[1].split('.nii.gz')[0]
                mask_names.append(mask_name)

                # resample mask
                preprocess.resample_mask(mask_name, fname_mask)

        np.save(f'../dataA/preprocessed/{patient_id}/mask_names', mask_names)

preprocess_patient_dataA()

## Run algorithms

In [ ]:
#from patients.A_b5_hyperparams_supervised import net_pars  # uncomment if using 5 b-values
from patients.A_b4_hyperparams_supervised import net_pars  # uncomment if using 4 b-values

def run_algorithms():
    # user specific parameters
    patient_ids = ['EMIN_1001', 'EMIN_1005', 'EMIN_1007', 'EMIN_1011', 'EMIN_1019',
                   'EMIN_1020', 'EMIN_1022', 'EMIN_1032', 'EMIN_1038', 'EMIN_1042', 
                   'EMIN_1044', 'EMIN_1045', 'EMIN_1048', 'EMIN_1055', 'EMIN_1057', 
                   'EMIN_1060', 'EMIN_1064', 'EMIN_1066', 'EMIN_1068', 'EMIN_1075', 
                   'EMIN_1077', 'EMIN_1079', 'EMIN_1081', 'EMIN_1084', 'EMIN_1086',
                   'EMIN_1090', 'EMIN_1092', 'EMIN_1093', 'EMIN_1096', 'EMIN_1097', 
                   'EMIN_1099']
    mask_root_names = ['GTVn']
    dataset_name = 'dataA'
    nums_bvals = [4, 5]
    training_method = 'a'
    average = 0
    #################################################################

    df_param_stats = pd.DataFrame(columns=['patient_id', 'num_bvals', 'method', 'param', 'param_val'])
    df_param_diffs_stats = pd.DataFrame(columns=['patient_id', 'num_bvals', 'method', 'param', 'param_val'])
    df_CV_stats = pd.DataFrame(columns=['patient_id', 'param', 'param_val'])
    for patient_id in patient_ids:

        # Run algorithms
        for num_bvals in nums_bvals:
            save_name = f'patientsA_LOO_{training_method}_b{num_bvals}'

            mask_names = []
            for mask_root_name in mask_root_names:
                fnames_mask = glob.glob(f'../dataA/raw/{patient_id}/Segmentation/mask_{mask_root_name}' + '*.nii.gz')
                for fname_mask in fnames_mask: 
                    mask_name = fname_mask.split(f'../dataA/raw/{patient_id}/Segmentation/mask_')[1].split('.nii.gz')[0]
                    mask_names.append(mask_name)

                    """
                    # lsq
                    ivim_lsq = from_patients.IVIM_fit(patient_id, 
                                                    mask_name, 
                                                    method='lsq', 
                                                    dataset_name=dataset_name,
                                                    num_bvals=num_bvals,
                                                    save_name=save_name)
                    ivim_lsq.fit_ivim()

                    # seg
                    ivim_seg = from_patients.IVIM_fit(patient_id, mask_name, method='seg',
                                                            dataset_name=dataset_name,
                                                            save_name=save_name)
                    ivim_seg.fit_ivim()
                    """
                

            #np.save(f'../dataA/preprocessed/{patient_id}/mask_names', mask_names)
            """
            # dnn, supervised
            dnn_supervised = from_patients.IVIM_dnn(patient_id, 
                                                    mask_names, 
                                                    learning='sup', 
                                                    dataset_name=dataset_name,
                                                    num_bvals=num_bvals,
                                                    training_method='a',
                                                    save_name=save_name)
            trained_model_dnn_supervised = torch.load(f'../simulations/simulations_data/b{num_bvals}/snr20/dnn_sup/r5_optim_snr20_d0428/trained_model_net_1')
            dnn_supervised.predict_ivim(trained_model_dnn_supervised)
            """
            """
            # dnn, self-supervised
            dnn_selfsupervised = from_patients.IVIM_dnn(patient_id, 
                                                        mask_names, 
                                                        learning='slf', 
                                                        dataset_name=dataset_name,
                                                        num_bvals=num_bvals,
                                                        training_method=training_method,
                                                        save_name=save_name)
            #trained_model_dnn_selfsupervised = dnn_selfsupervised.train_ivim()
            trained_model_dnn_selfsupervised = torch.load(f'../dataA/output/{save_name}/{patient_id}/dnn_slf/trained_model_net')
            dnn_selfsupervised.predict_ivim(trained_model_dnn_selfsupervised)
            """


        # Plotting
        for mask_name in mask_names:
            save_name_b5 = f'patientsA_LOO_{training_method}_b5'
            save_name_b4 = f'patientsA_LOO_{training_method}_b4'
            
            patientplt_ivim_b5 = from_patientplot.PatientpltIVIM(patient_id, mask_name, average=average, save_name = save_name_b5)
            patientplt_ivim_b5.plt_masked_param_map()
            patientplt_ivim_b5.plt_CV()
            patientplt_ivim_b5.plt_consistency()

            patientplt_ivim_b4 = from_patientplot.PatientpltIVIM(patient_id, mask_name, average=average, save_name = save_name_b4)
            patientplt_ivim_b4.plt_masked_param_map()
            patientplt_ivim_b4.plt_CV()
            patientplt_ivim_b4.plt_consistency()

            patientplt_ivim_b5_vs_b4 = from_patientplot.Patientplt_across_bvals(patient_id, mask_name, average=average, training_method=training_method, patientplt_b5=patientplt_ivim_b5, patientplt_b4=patientplt_ivim_b4)
            patientplt_ivim_b5_vs_b4.plt_diff_param_maps()

            df_param_stats = patientplt_ivim_b5_vs_b4.add_to_df_param_stats(df_param_stats)
            df_param_diffs_stats = patientplt_ivim_b5_vs_b4.add_to_df_param_diffs_stats(df_param_diffs_stats)
            df_CV_stats = patientplt_ivim_b5_vs_b4.add_to_df_CV_stats(df_CV_stats)


run_algorithms()


## Analysis

In [ ]:
training_method = 'b'


ppp = from_patient_populationA_plot.Patient_populationA_plot(training_method)
ppp.plot_ivim_params_stats()
ppp.plot_ivim_param_diffs_stats()
ppp.plot_CV_stats()

ppp.print_ivim_params_stats(plot_name='params', num_bvals='5 b-values')
ppp.print_ivim_params_stats(plot_name='params', num_bvals='4 b-values')

ppp.print_ivim_params_stats(plot_name='param_diffs', num_bvals='5 b-values')
ppp.print_ivim_params_stats(plot_name='param_diffs', num_bvals='4 b-values')

ppp.print_ivim_params_stats(plot_name='CV', num_bvals='5 b-values')
ppp.print_ivim_params_stats(plot_name='CV', num_bvals='4 b-values')

